# Homework 01

### Brown University  
### DATA 1010  
### Fall 2020

In [1]:
using CSV, DataFrames, Random

## Part I

In this problem, we'll explore the algorithmic side of a simple and common database algorithm.

In [2]:
# begin by loading the two local CSV files
df1 = CSV.read("flights1.csv")
df1 |> head

,id,date,delay,distance
,Int64,Int64,Int64,Int64
1,4222203651472,2120910,10,328
2,3154993796870,1141335,5,279
3,1290620327152,3131830,-7,293
4,6201397169412,1051910,22,255
5,9573467304590,3311247,6,229
6,2602364768170,1230615,4,197


In [3]:
df2 = CSV.read("flights2.csv")
df2 |> head

,id,origin,destination
,Int64,String,String
1,2140856843975,MCI,MDW
2,9172973311495,LAX,PHX
3,3113090932533,ONT,SMF
4,4130406938034,OAK,LAX
5,4550074644436,MSY,HOU
6,1477758908913,LAS,LAX


Each row of `df1` corresponds to a particular flight (with identifier `id`) and contains information about the date of the flight, how much it was delayed (in minutes), and the number of miles on the journey. The second database includes the origin and destination airport codes.

The problem we're looking to solve is to put these two data frames together, so that each row contains all five pieces of information for a given flight. This is called a *join* operation, and in real life you'll almost always use a library to do that operation. However, we're going to do it by hand as a learning experience and as a way of highlighting some of the aspects of the problem which are relevant even if you're using library code.

### Problem 1
Write a function to perform this join operation using the most naive algorithm: loop through the rows of the first dataframe. For each ID you encounter, use `findfirst` to search the `id` column of the second data frame to find the matching information.

*Hint*. You just need to know a couple things about data frames in Julia to do this problem: (1) You can access a data frame entry using the row index and the column name, like `df[3, "id"]` returns the `id` value for the third row of the data frame `df`. And (2) you can construct a data frame from an array of *named tuples*, like so:

In [4]:
(name = "Alice", age = 42)

(name = "Alice", age = 42)

In [5]:
typeof((name = "Alice", age = 42))

NamedTuple{(:name, :age),Tuple{String,Int64}}

In [6]:
DataFrame([
    (name = "Alice", age = 42), 
    (name = "Bob", age = 38)
])

,name,age
,String,Int64
1,Alice,42
2,Bob,38


(Note: in the interest of keeping the code very simple, we're going to use knowledge of the column names in the body of the function. For a generic implementation, we would need to read these column names from the data frames `df1` and `df2`.)

In [7]:
function join_naive(df1, df2)
    rows = []
    for i in 1:size(df1, 1)
        id = df1[i, "id"]
        matching_index = findfirst(isequal(id), df2[:, "id"])
        push!(rows, (
            id = id,
            date = df1[i, "date"],
            delay = df1[i, "delay"],
            distance = df1[i, "distance"],
            origin = df2[matching_index, "origin"], 
            destination = df2[matching_index, "destination"],
        ))
    end
    DataFrame(rows)
end
join_naive(df1, df2) |> head

,id,date,delay,distance,origin,destination
,Int64,Int64,Int64,Int64,String,String
1,4222203651472,2120910,10,328,ABQ,PHX
2,3154993796870,1141335,5,279,GEG,PDX
3,1290620327152,3131830,-7,293,LBB,DAL
4,6201397169412,1051910,22,255,BWI,RDU
5,9573467304590,3311247,6,229,STL,IND
6,2602364768170,1230615,4,197,FLL,TPA


This did work, but I noticed it was very slow to run. That makes sense, because for each of the couple hundred thousand rows in the first data frame, we have to search through typically tens of thousands of rows in the second data frame to find a match. 

### Problem 2
Write a function to perform this join operation using a different approach: build a *dictionary* which maps each flight ID value to the corresponding row of the second table. Then loop through the rows of the first table, using the dictionary to access the relevant data from the second table. 

In [8]:
function join_dict(df1, df2)
    id_mapping = Dict(df2[i, "id"] => i for i in 1:size(df2, 1))
    rows = []
    for i in 1:size(df1, 1)
        id = df1[i, "id"]
        matching_index = id_mapping[id] # this is the line that's different!
        push!(rows, (
            id = id,
            date = df1[i, "date"],
            delay = df1[i, "delay"],
            distance = df1[i, "distance"],
            origin = df2[matching_index, "origin"], 
            destination = df2[matching_index, "destination"],
        ))
    end
    DataFrame(rows)
end
join_dict(df1, df2) |> head

,id,date,delay,distance,origin,destination
,Int64,Int64,Int64,Int64,String,String
1,4222203651472,2120910,10,328,ABQ,PHX
2,3154993796870,1141335,5,279,GEG,PDX
3,1290620327152,3131830,-7,293,LBB,DAL
4,6201397169412,1051910,22,255,BWI,RDU
5,9573467304590,3311247,6,229,STL,IND
6,2602364768170,1230615,4,197,FLL,TPA


We can see that we get the same result, but this was *much* snappier. Clearly whatever Julia's `Dict` object is doing to look up its keys, it's doing it in some way that's much faster than `findfirst`. 

### Problem 3
Time your functions from (a) and (b); which is faster? Here's an example of how to time a simple function in Julia:

In [9]:
@time join_naive(df1, df2);

196.471863 seconds (7.77 M allocations: 398.114 GiB, 15.38% gc time)


In [10]:
@time join_dict(df1, df2);

  0.674085 seconds (8.69 M allocations: 276.704 MiB, 9.65% gc time)


### Problem 4
Now suppose that you get to store the dictionary mapping ids to rows *in advance*. Rewrite your function in (b) in such a way that it takes that dictionary as an argument. Time the evaluation of this new function (which gets the dictionary as an argument rather than having to evaluate it). How much faster is it?

We can investigate how much time to dictionary construction is taking by simply moving it out of the function's body and supplying it as an argument: 

In [11]:
function join_with_index(df1, df2, id_mapping)
    rows = []
    for i in 1:size(df1, 1)
        id = df1[i, "id"]
        matching_index = id_mapping[id] # this is the line that's different!
        push!(rows, (
            id = id,
            date = df1[i, "date"],
            delay = df1[i, "delay"],
            distance = df1[i, "distance"],
            origin = df2[matching_index, "origin"], 
            destination = df2[matching_index, "destination"],
        ))
    end
    DataFrame(rows)
end

id_mapping = Dict(df2[i, "id"] => i for i in 1:size(df2, 1))
@time join_with_index(df1, df2, id_mapping);

  0.634052 seconds (6.26 M allocations: 217.911 MiB, 11.17% gc time)


### Problem 5
What would be another way we could make this operation faster without indexing (that's what the dictionary storing operation in (d) is called)?

## Part II

### Problem 6

In the game of Set, every card has four features:

**Number**. 1, 2, or 3  
**Color**. purple, red, or green  
**Shape**. oval, squiggle, or diamond  
**Shading**. striped, solid, or outline

<img src="set.png"/>

There is exactly one card for every possible combination of attributes (for example there's exactly one card with 2 red solid squiggles), so there are 81 cards in total. 

Three cards are said to form a set if for each feature, the three cards are either all the same or all different. For example, the cards shown above form a set because their shapes are all different, their shading is all different, the numbers are all different, and the colors are all different. 

The cards are shuffled, and 12 cards are dealt face up for all players to see. If any three of the cards forms a set, then the first player to identify a set gets to pick up those cards, and the cards are replaced from the deck. If no sets are present, then three additional cards are dealt. If a set is identified at 15 cards, then the three removed cards are not replaced, and the count goes back down to 12. If no sets are present even at 15 cards, then three more cards are dealt to get up to 18, and so on. The game ends when the deck is empty and there are no more sets. 

Write a program to simulate this game, play it 10,000 times, and find the proportion of games in which 18 cards appear at some point.

**Hints:**

(1) You should represent cards using a new `Card` type, rather than using 4-tuples of `Int`s.  
(2) It's going to be important to break your program up into small, dedicated functions. For example, you can write a function for returning the 81 total cards, a function for shuffling the deck, a function which takes three cards and returns true or false depending on whether they form a set, a function for dealing cards, a function for finding all the sets on the board, a function which takes a single turn by randomly selecting one of the available sets or adding three cards to the board if there are no sets, and finally a function which puts those functions together to play the game.  
(3) Some Julia tips: 
  - `randperm` generates random permutations, and you can index an array with the resulting list to shuffle it. 
  - `vcat` concatenates two arrays.
  - You probably want to use a four-dimensional array comprehension to generate the list of all cards, and you can index it with a colon to flatten it.
  - To remove elements at positions `i`, `j`, and `k` from an array `A`, you can do `A[setdiff(1:end, (i,j,k))]`
  - You can start with an empty array and grow it one element at a time inside a loop using this pattern:
    ```julia 
    A = []
    for x in 1:10
        push!(A, x)
    end
    ```
  - You can use nested `for` loops to check all the triples to look for a set. There are more efficient algorithms, but that's OK.

Your solution should contain working code, but you should also discuss in prose how each function works and how the functions fit together in the design of the program.

*Solution*. 

Next we arrive at the most interesting part of the algorithm: how to find sets on the board. We're going to do this in a naive way: loop through every triple of cards and push to an array of matches whenever a match is found:

Finally, we can investigate the question of how often 18 cards appear on the board. We'll use a comprehension to play the game a thousand times, and we'll use `countmap` to ascertain how many times each maximum card count occurred. 

## Part III

### Problem 7

Some things which look like functions in Julia start with `@`; these are called **macros**. The difference between a macro and a function is that the former is able to operate on the *code expressions* supplied to it, while a function can only operate on the *values* that those code expressions evaluate to. 

For example, if `f` is a function, then `f(2)` is always equivalent to `f(1 + 1)`. However, macros can behave differently depending on the expression: 

In [12]:
x = 1
@show(x + 1);

x + 1 = 2


In [13]:
@show(1 + 1);

1 + 1 = 2


In [14]:
@show(2);

2 = 2


What would you need to do if you wanted to be able to use plain functions to do a computation with the supplied expression (not just the value it evaluates to)? In other words, write a function `showfunction` which achieves an effect somewhat similar to the `@show` macro. 

Feel free to look up whatever you want online to solve this problem. You might want to look at the `Meta.parse` function (no import necessary), which turns a string into an expression, as well as the `eval` function, which evaluates an expression.

(Note that this technique is essential in a language like Python, which doesn't have macros. By contrast, R's approach is at the opposite end of the spectrum: [*every* function gets access to the code of its arguments](http://adv-r.had.co.nz/Computing-on-the-language.html).)

(Also note: this exercise is helpful because it sheds some light on one way that language design influences the interfaces of various data science libraries you will use.)

### Problem 8

Explain how the following function returns a value even though it contains no return statement. 

In [15]:
function A(x)
    if x ≥ 0
        x
    else
        -x
    end
end

using Test
@testset begin
    @test A(-3) == 3
    @test A(0.0) == 0.0
    @test A(1) == 1
end

Test Summary: | Pass  Total
test set      |    3      3


Test.DefaultTestSet("test set", Any[], 3, false)

As in Problem 7, feel free to look up whatever you want and write a brief explanation of what's going on.